In [2]:
!pip install LambdaRankNN
!pip install pyltr

In [3]:
from google.colab import drive,files
from google.colab import drive,files
drive.mount('/content/drive')

import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import GridSearchCV


#Lambdaranknn library for lambdarank and ranknet in python
from LambdaRankNN import RankNetNN, LambdaRankNN
#Pyltr library for lambdamart in python
import pyltr

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import CyclicLR

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def load_data(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        x, y, q = [], [], []
        info = []
        for line in lines:
            feature, comment = line.split(' #')
            feature = feature.split()
            yi = feature[0]
            qi = feature[1].split(':')[-1]
            xi = feature[2:]
            xi = [f.split(':')[-1] for f in xi]
            x.append(xi)
            y.append(yi)
            q.append(qi)
            info.append(comment)
        return np.array(x, dtype=np.float32), np.array(y, dtype=np.int), np.array(q, dtype=np.int)

# rankNet


In [ ]:
# Reading dataset
train_X, train_y, train_queries = load_data("/content/drive/MyDrive/IR Package/train.txt")
test_X, test_y, test_queries = load_data("/content/drive/MyDrive/IR Package/test.txt")
print("Data Loaded")
print(train_X.shape, train_y.shape)

# Training RankNetNN model
ranker = RankNetNN(input_size=train_X.shape[1], 
                   hidden_layer_sizes=(16,8,), 
                   activation=('relu', 'relu',), 
                   solver='adam')
epochs = 200
ndcg_at = 10
ranker.fit(train_X, train_y, train_queries, epochs= epochs)
train_y_pred = ranker.predict(train_X)
ranker.evaluate(train_X, train_y, train_queries, eval_at=ndcg_at)
test_y_pred = ranker.predict(test_X)
ranker.evaluate(test_X, test_y, test_queries, eval_at=ndcg_at)


df1 = pd.DataFrame(train_y_pred)
df3 = pd.DataFrame(test_y_pred)

ranknet = pd.concat([df1, df3], axis = 1)
ranknet.columns = ['Train_pred', 'Test_pred']
ranknet.to_csv("ranknet.csv", index = False)
#files.download('ranknet.csv')

Data Loaded
(42158, 46) (42158,)
Epoch 1/10
7688/7688 [==============================] - 9s 1ms/step - loss: 0.5515
Epoch 2/10
7688/7688 [==============================] - 9s 1ms/step - loss: 0.5294
Epoch 3/10
7688/7688 [==============================] - 9s 1ms/step - loss: 0.5207
Epoch 4/10
7688/7688 [==============================] - 9s 1ms/step - loss: 0.5146
Epoch 5/10
7688/7688 [==============================] - 9s 1ms/step - loss: 0.5099
Epoch 6/10
7688/7688 [==============================] - 9s 1ms/step - loss: 0.5056
Epoch 7/10
7688/7688 [==============================] - 9s 1ms/step - loss: 0.5024
Epoch 8/10
7688/7688 [==============================] - 9s 1ms/step - loss: 0.5003
Epoch 9/10
7688/7688 [==============================] - 9s 1ms/step - loss: 0.4982
Epoch 10/10
7688/7688 [==============================] - 9s 1ms/step - loss: 0.4961
ndcg: 0.6115024913740011
ndcg@10: 0.46909315006207725
ndcg@10: 0.4748094691564848


# LambdaMart


In [ ]:

with open( "/content/drive/MyDrive/IR Package/train.txt") as trainfile, \
    open("/content/drive/MyDrive/IR Package/test.txt") as valifile, \
    open("/content/drive/MyDrive/IR Package/test.txt") as evalfile:
  TX, Ty, Tqids, _ = pyltr.data.letor.read_dataset(trainfile)
  VX, Vy, Vqids, _ = pyltr.data.letor.read_dataset(valifile)
  EX, Ey, Eqids, _ = pyltr.data.letor.read_dataset(evalfile)

metric = pyltr.metrics.NDCG(k=10)
monitor = pyltr.models.monitors.ValidationMonitor(
    VX, Vy, Vqids, metric=metric, stop_after=100)
"""
mart = pyltr.models.LambdaMART(
    metric=metric, 
    query_subsample=0.5,
    max_leaf_nodes=10,
    min_samples_leaf=64,
    verbose=1)
params = {'n_estimators':[500, 1000, 2000, 5000], 'learning_rate':[0.01, 0.02, 0.1, 0.2, 0.5], 'subsample':[0.5,1], 'max_features':[0.5,'auto']}
grid = GridSearchCV(mart, params)
grid.fit(TX, Ty, Tqids, monitor=monitor)
print("Grid search results")
print(grid.cv_results_.keys())
"""
ranker2 = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=1000,
    learning_rate=0.01,
    max_features=0.5,
    query_subsample=0.5,
    max_leaf_nodes=10,
    min_samples_leaf=16,
    verbose=1,
)

ranker2.fit(TX, Ty, Tqids, monitor=monitor)
train_y_pred = ranker2.predict(TX)
print('Train NDCG@10:', metric.calc_mean(Tqids, Ty, train_y_pred))

test_y_pred = ranker2.predict(EX)
print('Test NDCG@10:', metric.calc_mean(Eqids, Ey, test_y_pred))


df1 = pd.DataFrame(train_y_pred)
df3 = pd.DataFrame(test_y_pred)
mart = pd.concat([df1, df3], axis = 1)
mart.columns = ['Train_pred',  'Test_pred']
mart.to_csv("lambda_mart.csv", index = False)
#files.download('lambda_mart.csv')

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.2585       0.2417       15.46m      C:      0.2700 B:      0.2700 S:  0
    2       0.3491       0.0848       15.29m      C:      0.3785 B:      0.3785 S:  0
    3       0.3788       0.0184       15.47m      C:      0.3960 B:      0.3960 S:  0
    4       0.3606       0.0097       15.49m      C:      0.4026 B:      0.4026 S:  0
    5       0.3815       0.0048       15.48m      C:      0.4134 B:      0.4134 S:  0
    6       0.3984       0.0015       15.50m      C:      0.4166 B:      0.4166 S:  0
    7       0.3962       0.0090       15.42m      C:      0.4274 B:      0.4274 S:  0
    8       0.3829       0.0033       15.37m      C:      0.4297 B:      0.4297 S:  0
    9       0.3921       0.0023       15.40m      C:      0.4312 B:      0.4312 S:  0
   10       0.3923      -0.0003       15.41m      C:      0.4327 B:      0.4327 S:  0
   15       0.4218       0.0015       15.34m      C: 

# Lambda rank

In [ ]:
#Reading Dataset
train_X, train_y, train_queries = load_data("/content/drive/MyDrive/IR Package/train.txt")
test_X, test_y, test_queries = load_data("/content/drive/MyDrive/IR Package/test.txt")
print("Loaded data")

print(train_X.shape, train_y.shape)

# Training LambdaRankNN model
ranker3 = LambdaRankNN(input_size=train_X.shape[1], 
                      hidden_layer_sizes=(16,8,), 
                      activation=('relu', 'relu',), 
                      solver='adam')
epochs = 200
ndcg_at = 10
ranker3.fit(train_X, train_y, train_queries, epochs=epochs)

train_y_pred = ranker3.predict(train_X)
print("Train evaluate")
ranker3.evaluate(train_X, train_y, train_queries, eval_at=ndcg_at)
test_y_pred = ranker3.predict(test_X)
print("Test evaluate")
ranker.evaluate(test_X, test_y, test_queries, eval_at=ndcg_at)

df1 = pd.DataFrame(train_y_pred)
df3 = pd.DataFrame(test_y_pred)

lambdarank = pd.concat([df1, df3], axis = 1)
lambdarank.columns = ['Train_pred', 'Test_pred']
lambdarank.to_csv("lambda_rank.csv", index = False)
files.download('lambda_rank.csv')

Loaded data
(42158, 46) (42158,)
Epoch 1/200
7688/7688 [==============================] - 9s 1ms/step - loss: 0.0120
Epoch 2/200
7688/7688 [==============================] - 9s 1ms/step - loss: 0.0114
Epoch 3/200
7688/7688 [==============================] - 9s 1ms/step - loss: 0.0110
Epoch 4/200
7688/7688 [==============================] - 9s 1ms/step - loss: 0.0107
Epoch 5/200
7688/7688 [==============================] - 9s 1ms/step - loss: 0.0105
Epoch 6/200
7688/7688 [==============================] - 9s 1ms/step - loss: 0.0104
Epoch 7/200
7688/7688 [==============================] - 9s 1ms/step - loss: 0.0102
Epoch 8/200
7688/7688 [==============================] - 9s 1ms/step - loss: 0.0101
Epoch 9/200
7688/7688 [==============================] - 9s 1ms/step - loss: 0.0101
Epoch 10/200
7688/7688 [==============================] - 9s 1ms/step - loss: 0.0100
Epoch 11/200
7688/7688 [==============================] - 9s 1ms/step - loss: 0.0099
Epoch 12/200
7688/7688 [=================

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


# Ensemble of Ranking Algorithms



In [ ]:
seed = 25
torch.manual_seed(seed)

def read_data(filename):
    y1 = pd.read_csv(filename)['Test_pred']
    y1 = y1.dropna()
    y1 = torch.tensor(y1).unsqueeze(1)
    return y1

print("Data Loading")  
y1 = read_data("lambda_mart.csv")
y2 = read_data("lambda_rank.csv")
y3 = read_data("ranknet.csv")

print(y1.shape, y2.shape, y3.shape)
input_val = torch.cat([y1, y2, y3], dim = 1)
print("input_val", input_val.shape)

with open("/content/drive/MyDrive/IR Package/test.txt") as evalfile:
  _, Ey, Eqids, _ = pyltr.data.letor.read_dataset(evalfile)

print("Data Loading Complete")
targets = torch.tensor(Ey).unsqueeze(1)
    
class FeedForwardNeuralNet(nn.Module):
    def __init__(self, inp, out, hdim1, hdim2):
        super().__init__()
        self.fc1 = nn.Linear(inp, hdim1)
        self.fc2 = nn.Linear(hdim1, hdim2)
        self.fc3 = nn.Linear(hdim2, out)
        self.relu = nn.ReLU(inplace = True)
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.normal_(self.fc1.bias)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.normal_(self.fc2.bias)
        nn.init.xavier_uniform_(self.fc3.weight)
        nn.init.normal_(self.fc3.bias)
        
        
    def forward(self, input_val):
        x = self.fc1(input_val)
        x = self.relu(x)
        x = self.relu(self.fc2(x))
        out = self.fc3(x)
        return out
    
# Setting device type
if torch.cuda.is_available():
    print("Running on GPU")
    device = torch.device("cuda:2")
else:
    device = torch.device("cpu")
    
print("Running on device type: {}".format(device))

num_epochs = 2000
batch_size = 30000
learning_rate = 1e-2


model = FeedForwardNeuralNet(3,1,8,8)
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = CyclicLR(optimizer, base_lr=learning_rate, max_lr=1, mode="exp_range", gamma=0.999)

criterion = nn.MSELoss()
training_loss_list = []

model.train()
for epoch in range(num_epochs):
    training_loss = 0
    for i in range(0,len(input_val), batch_size):
        start = i
        end = start + batch_size
        inputs = input_val[start:end,:].float()
        target = targets[start:end].float()
        inputs = inputs.to(device)
        target = target.to(device)
        out = model(inputs)
        optimizer.zero_grad()
        loss = torch.sqrt(criterion(out, target))
        training_loss += loss.item()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
    scheduler.step(training_loss)
    print("epoch", epoch, "training_loss_per_epoch", training_loss)
    training_loss_list.append(training_loss)

metric = pyltr.metrics.NDCG(k=10)
target1 = target.numpy().reshape(target.shape[0],)
out_r = out.detach().numpy().reshape(out.shape[0],)
print("NDCG Ensemble", metric.calc_mean(Eqids, target1, out_r))

Data Loading
torch.Size([13652, 1]) torch.Size([13652, 1]) torch.Size([13652, 1])
input_val torch.Size([13652, 3])
Data Loading Complete
Running on device type: cpu
epoch 0 training_loss_per_epoch 1.672982096672058
epoch 1 training_loss_per_epoch 1.6266167163848877
epoch 2 training_loss_per_epoch 1.533496379852295
epoch 3 training_loss_per_epoch 1.4065755605697632
epoch 4 training_loss_per_epoch 1.256514549255371
epoch 5 training_loss_per_epoch 1.0961060523986816
epoch 6 training_loss_per_epoch 0.9418767094612122
epoch 7 training_loss_per_epoch 0.8267958164215088
epoch 8 training_loss_per_epoch 0.75444096326828
epoch 9 training_loss_per_epoch 0.7086610794067383
epoch 10 training_loss_per_epoch 0.6860628128051758
epoch 11 training_loss_per_epoch 0.6818895936012268
epoch 12 training_loss_per_epoch 0.6836795210838318
epoch 13 training_loss_per_epoch 0.6826696395874023
epoch 14 training_loss_per_epoch 0.6751812100410461
epoch 15 training_loss_per_epoch 0.6609942317008972
epoch 16 training_

# References



1.   Burges, Christopher JC. "From ranknet to lambdarank to lambdamart: An overview." Learning 11.23-581 (2010): 81.
2. Kuzi, Saar, et al. "Analysis of Adaptive Training for Learning to Rank in Information Retrieval." Proceedings of the 28th ACM International Conference on Information and Knowledge Management. 2019.
3.   https://towardsdatascience.com/how-to-code-a-simple-neural-network-in-pytorch-for-absolute-beginners-8f5209c50f


